### 추천시스템(아이템 기반 협업 필터링)

In [1]:
#협업 필터링 방식 - 사용자 기반, 아이템 기반(주로 아이템 기반 필터링을 사용함)

# 사용자 기반 협업 필터링:
#   사용자의 행동 양식을 기반으로 추천(평점, 상품 구매 이력 등)
#   x축에 아이템, y축에 사용자를 표시
#   어떤 사용자와 비슷한 성향의 고객들이 구매한 상품을 추천

# 아이템 기반 협업 필터링(Item based collaborative Filtering):
#   x축에 사용자, y축에 아이템을 표시하고 유사한 아이템을 추천해 주는 방식
#   주로 코사인 유사도를 사용하여 계산함
#   어떤 상품과 비슷한 상품을 추천

import pandas as pd

df = pd.read_csv('c:/workspace3/data/ratings_small.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [2]:
df = df.pivot_table('rating', index = 'userId', columns = 'movieId')
df

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#x축 movieId, y축 userId
#사용자 아이디별로 영화에 대한 평점을 볼 수 있음
#영화 제목을 가져오기 위해 데이터프레임 조인

df_ratings= pd.read_csv('c:/workspace3/data/ratings_small.csv')
df_movies = pd.read_csv('c:/workspace3/data/tmdb_5000_movies.csv')
df_movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [5]:
# 필드명 id를 movieId로 변경

df_movies.rename(columns = {'id': 'movieId'}, inplace = True)

In [12]:
# ratings의 movieId와 movies의 movieId가 같은 행끼리 조인

df_ratings_movies = pd.merge(df_ratings, df_movies, on = 'movieId')
# 영화코드와 영화제목을 결합

df_ratings_movies

,userId,movieId,rating,timestamp,budget,genres,homepage,keywords,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,1,2105,4.0,1260759139,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296
1,4,2105,4.0,949896114,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296
2,15,2105,4.0,1052896867,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296
3,30,2105,2.0,994439964,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296
4,34,2105,4.0,973747765,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18566,652,26843,5.0,1440269953,35000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 36, ""nam...",NaN,"[{""id"": 131, ""name"": ""italy""}, {""id"": 539, ""na...",en,Lion of the Desert,...,"[{""iso_3166_1"": ""LY"", ""name"": ""Libyan Arab Jam...",1981-04-17,0,173.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,He was a man of honor in a war without any.,Lion of the Desert,7.4,34
18567,652,103731,5.0,1439586954,10000000,"[{""id"": 18, ""name"": ""Drama""}]",NaN,"[{""id"": 1261, ""name"": ""river""}, {""id"": 1585, ""...",en,Mud,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-05-01,21587700,130.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Mud,7.0,1068
18568,659,167,4.0,836137550,48000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 878, ""nam...",http://www.k-pax.com/,"[{""id"": 642, ""name"": ""robbery""}, {""id"": 1566, ...",en,K-PAX,...,"[{""iso_3166_1"": ""DE"", ""name"": ""Germany""}, {""is...",2001-10-22,50315140,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Change the way you look at the world.,K-PAX,7.1,695
18569,659,563,3.0,834694187,105000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...",NaN,"[{""id"": 305, ""name"": ""moon""}, {""id"": 1423, ""na...",en,Starship Troopers,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1997-11-06,121214377,129.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The only good bug is a dead bug.,Starship Troopers,6.7,1560


In [7]:
# x축 사용자아이디, y축 영화제목의 피벗테이블을 만들고
# NaN은 0으로 채움

df2 = df_ratings_movies.pivot_table('rating', index = 'userId', columns = 'title').fillna(0)
df2

title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2046,21 Grams,25th Hour,...,Willy Wonka & the Chocolate Factory,World Trade Center,X-Men Origins: Wolverine,Y Tu Mamá También,You Only Live Twice,"You, Me and Dupree",Young Frankenstein,Zodiac,eXistenZ,xXx
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#아이템 기반 협업 필터링에서는 row가 아이템이어야 하므로 x,y축을 바꿈

df2 = df2.transpose()
df2

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15 Minutes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Blocks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"You, Me and Dupree",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Young Frankenstein,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
Zodiac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.metrics.pairwise import cosine_similarity

#코사인 유사도 계산
movie_sim = cosine_similarity(df2, df2)
print(movie_sim.shape)

(856, 856)


In [10]:
#코사인유사도 벡터를 데이터프레임으로 변환

df_movie_sim = pd.DataFrame(data = movie_sim, index = df2.index, columns = df2.index)
df_movie_sim

title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2046,21 Grams,25th Hour,...,Willy Wonka & the Chocolate Factory,World Trade Center,X-Men Origins: Wolverine,Y Tu Mamá También,You Only Live Twice,"You, Me and Dupree",Young Frankenstein,Zodiac,eXistenZ,xXx
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,1.000000,0.0,0.000000,0.182153,0.0,0.022069,0.085323,0.0,0.00000,0.103490,...,0.059856,0.0,0.161801,0.088076,0.0,0.0,0.097588,0.000000,0.000000,0.014121
12 Angry Men,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1408,0.000000,0.0,1.000000,0.447214,0.0,0.173381,0.028245,0.0,0.00000,0.000000,...,0.146955,0.0,0.148968,0.140265,0.0,0.0,0.191675,0.000000,0.000000,0.000000
15 Minutes,0.182153,0.0,0.447214,1.000000,0.0,0.077538,0.050526,0.0,0.00000,0.129863,...,0.197160,0.0,0.216516,0.141138,0.0,0.0,0.085720,0.115684,0.121365,0.000000
16 Blocks,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.00000,0.000000,...,0.000000,0.0,0.130347,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"You, Me and Dupree",0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.000000
Young Frankenstein,0.097588,0.0,0.191675,0.085720,0.0,0.204590,0.115720,0.0,0.00000,0.000000,...,0.230622,0.0,0.423840,0.236086,0.0,0.0,1.000000,0.214856,0.110536,0.204346
Zodiac,0.000000,0.0,0.000000,0.115684,0.0,0.014016,0.222842,0.0,0.00000,0.075115,...,0.359021,0.0,0.288208,0.201826,0.0,0.0,0.214856,1.000000,0.163801,0.105379


In [11]:
#킹콩 영화와 코사인 유사도가 비슷한 5개의 영화 추천

df_movie_sim["King Kong"].sort_values(ascending=False)[1:5]

title
Fantasia     0.648886
2046         0.648886
Liar Liar    0.486664
Rendition    0.486664
Name: King Kong, dtype: float64